This notebook is part of the *orix* documentation https://orix.readthedocs.io. Links to the documentation won’t work from the notebook.

# Visualizing Crystal Poles in the Pole Density Function

This notebook demonstrates how to quantify the distribution of crystallographic poles,
which is useful, for example, in texture analysis.

First, we load some sample orientations from a Titanium sample dataset which represent
crystal orientations in the sample reference frame:

NB. if not previously downloaded this will download some example data from an online
repository to a local cache, the docstring of [ti_orientations](reference.rst#orix.data.ti_orientations)
for more details):

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from orix.crystal_map import Phase
from orix.data import ti_orientations
from orix import plot
from orix.quaternion import symmetry
from orix.sampling.S2_sampling import sample_S2_equal_area_mesh
from orix.vector import Miller, Vector3d

ori = ti_orientations(allow_download=True)
ori.shape

Let's look at the sample's ${011}$ texture plotted in the stereographic projection.

First we must define the crystal's point group and generate the set of symmetrically
unique $(011)$ poles: 

In [ ]:
m = Miller(hkl=(0, 1, 1), phase=Phase(point_group=symmetry.D6))
m = m.symmetrise().unique()
m

Now let's compute the direction of these poles in the sample reference frame.

This is done using the [Orientation](reference.rst#orix.quaternion.Orientation)-[Vector3d](reference.rst#orix.vector.Vector3d)
`outer` product. We can pass `lazy=True` parameter to perform the computation in chunks 
using `Dask`, this helps to reduce memory usage when there are many computations to be
performed. 

In [ ]:
poles = (~ori).outer(m, lazy=True, progressbar=True, chunk_size=2000)
poles.shape

We can plot these poles in the sterographic projection:

In [ ]:
fig = poles.scatter(hemisphere='both', s=2, alpha=0.05, return_figure=True)

w, h = plt.rcParams['figure.figsize']
fig.set_figwidth(2 * h)
fig.set_figheight(h)

In this case there are many individual data points, which makes it difficult to
interpret whether regions contain higher or lower pole density.

In this case we can use the [Vector3d.pole_density_function()](reference.rst#orix.vector.Vector3d.pole_density_function)
to measure the pole density on the unit sphere $S_2$. Internally this uses the equal
area parameterization to calculate cells on $S_2$ with the same solid angle. In this
representation randomly oriented vectors have the same probability of intercepting each
cell, thus we can represent our sample's PDF as Multiples of Random Density (MRD). This
follows the work of (<cite data-cite="rohrer2004distribution">Rohrer et al.(2004)</cite>).

Below is the equal area sampling representation on $S_2$ in both the stereographic
projection and 3D, with a resolution of 10&deg;:

In [ ]:
fig = plt.figure(figsize=(3 * h, h))
ax0 = fig.add_subplot(131, projection='stereographic')
ax1 = fig.add_subplot(132, projection='stereographic')
ax2 = fig.add_subplot(133, projection='3d')

v_mesh = sample_S2_equal_area_mesh(resolution=10)

ax0.hemisphere = 'upper'
ax0.scatter(v_mesh)
ax0.show_hemisphere_label()
ax0.set_labels('x', 'y', None)

ax1.hemisphere = 'lower'
ax1.scatter(v_mesh)
ax1.show_hemisphere_label()
ax0.set_labels('x', 'y', None)

ax2.scatter(*v_mesh.data.T)

lim = 1
ax2.set_xlim(-lim, lim)
ax2.set_ylim(-lim, lim)
ax2.set_zlim(-lim, lim)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('z')
ax2.set_box_aspect((1, 1, 1))

fig.tight_layout()

For randomly distributed vectors on $S_2$, we can can see that MRD tends to 1 with an increasing number of vectors:

NB. PDF plots are displayed on the same color scale.

In [ ]:
num = (10_000, 100_000, 1_000_000, 10_000_000)

fig, ax = plt.subplots(ncols=4, figsize=(len(num) * h, h), subplot_kw=dict(projection='stereographic'))

for i, n in enumerate(num):
    v = Vector3d(np.random.randn(n, 3)).unit
    ax[i].pole_density_function(v.azimuth, v.polar, log=False, vmin=0.8, vmax=1.2)
    ax[i].set_labels('x', 'y', None)
    ax[i].set_title(str(n))

We can also change the sampling angular `resolution` on $S_2$, the colormap with the
`cmap` parameter, and broadening of the density distrbution with `sigma`:

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(3 * h, h), subplot_kw=dict(projection='stereographic'))

v = Vector3d(np.random.randn(1_000_000, 3)).unit

ax[0].pole_density_function(v.azimuth, v.polar, log=False, resolution=1)
ax[0].set_title('Sampling resolution: 1$\degree$')

# change sampling resolution on S2
ax[1].pole_density_function(v.azimuth, v.polar, log=False, resolution=5)
ax[1].set_title('Sampling resolution: 5$\degree$')

# increase peak broadening
ax[2].pole_density_function(v.azimuth, v.polar, log=False, resolution=1, sigma=15)
ax[2].set_title('Sampling resolution: 1$\degree$, $\sigma$: 15$\degree$')

for a in ax:
    a.set_labels('x', 'y', None)

fig.tight_layout()

Poles from real samples tend to not be randomly oriented, as the material microstructure
is arranged into regions of similar crystal orientation, known as grains.

The PDF for the measured ${011}$ poles from the Titanium sample loaded at the beginning
of the notebook:

In [ ]:
fig = poles.pole_density_function(hemisphere='both', return_figure=True, log=False)
fig.set_figwidth(2 * h)
fig.set_figheight(h)

We can also plot these densities on a `log` scale to reduce the contrast between high
and low density regions.

By comparing the point data shown at the top of the notebook with the calculated pole
densities from PDF, we can see that not all regions in the point data representation
have the same density and that PDF is needed for better quantification:

In [ ]:
fig, ax = plt.subplots(ncols=3, subplot_kw=dict(projection='stereographic'), figsize=(3 * h, h))

ax[0].hemisphere = 'upper'
ax[1].hemisphere = 'upper'

ax[0].scatter(poles, s=2, alpha=0.05)
ax[1].pole_density_function(poles.azimuth, poles.polar, log=True)
ax[2].scatter(poles, s=2, alpha=0.01, c='w')
ax[2].pole_density_function(poles.azimuth, poles.polar, log=True)

for a in ax:
    a.set_labels('x', 'y', None)